# 046_scrapy_当当网管道封装

接上一节

In [ ]:
# scrapy_dangdang_045/scrapy_dangdang_045/spiders/dang.py
import scrapy
from scrapy_dangdang_045.items import ScrapyDangdang045Item

class DangSpider(scrapy.Spider):
    name = "dang"
    allowed_domains = ["category.dangdang.com"]
    start_urls = ["https://category.dangdang.com/cp01.01.02.00.00.00.html"]

    def parse(self, response):
        # pipelines.py    下载数据
        # items.py``      定义数据结构

        # src = //ul[@id="component_59"]/li//img/@src
        # name = //ul[@id="component_59"]/li//img/@alt
        # price = //ul[@id="component_59"]/li//p[@class="price"]/span[1]/text() # 第一个span
        
        # 所有的selector的对象都可以再次调用xpath方法
        li_list = response.xpath('//ul[@id="component_59"]/li')
        for li in li_list:
            # 图片有懒加载，这里要用data-original 而不是src（有的是src2）
            src = li.xpath('.//img/@data-original').extract_first()
            # 第一张图片和其他的图片标签属性不一样
            # 第一张图片的src是可以使用的  其他的图片的地址是data-original
            if src:
                src = src
            else:
                src = li.xpath('.//img/@src').extract_first()

            name = li.xpath('.//img/@alt').extract_first()
            price = li.xpath('.//p[@class="price"]/span[1]/text()').extract_first()
            print(src, name, price)

            # items.py定义的数据结构
            book = ScrapyDangdang045Item(src=src, name=name, price=price)

            # 获取一个book就将book交给pipelines
            yield book

In [ ]:
# scrapy_dangdang_045/scrapy_dangdang_045/items.py

# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class ScrapyDangdang045Item(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    # 通俗的说就是你要下载的数据都有什么

    # 图片
    src = scrapy.Field()
    # 书名
    name = scrapy.Field()
    # 价格
    price = scrapy.Field()
    pass


In [ ]:
# scrapy_dangdang_045/scrapy_dangdang_045/pipelines.py

# 使用pipeline管道下载数据
# 首先要在settings.py中开启管道
# ------------------------------
# settings.py:
# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
# ITEM_PIPELINES = {
#     # 管道可以有很多个  管道是有优先级的    优先级的范围是1-1000 值越小优先级越高
#    "scrapy_dangdang_045.pipelines.ScrapyDangdang045Pipeline": 300,
# }



# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
from itemadapter import ItemAdapter

# 如果想使用管道的话 那么就必须在settings中开启管道
class ScrapyDangdang045Pipeline:

    # item就是yield后面的book对象
    def process_item(self, item, spider):
        # 以下这种模式不推荐    因为每传递过来一个对象 就会打开一次文件 对文件的操作过于频繁
        # (1) write 方法必须要写一个字符串，而不能是其他对象
        # (2) w模式 会每个对象都打开一次文件 覆盖之前的内容
        with open('book.json', 'a', encoding='utf-8') as fp:
            fp.write(str(item))
        return item

# 推荐使用下面的方法，避免频繁打开文件

In [ ]:
# scrapy_dangdang_045/scrapy_dangdang_045/pipelines.py

# 使用pipeline管道下载数据
# 首先要在settings.py中开启管道
# ------------------------------
# settings.py:
# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
# ITEM_PIPELINES = {
#     # 管道可以有很多个  管道是有优先级的    优先级的范围是1-1000 值越小优先级越高
#    "scrapy_dangdang_045.pipelines.ScrapyDangdang045Pipeline": 300,
# }



# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
from itemadapter import ItemAdapter

# 如果想使用管道的话 那么就必须在settings中开启管道
class ScrapyDangdang045Pipeline:
    # 在爬虫文件开始之前就执行的方法
    def open_spider(self, spider):
        print('++++++++++++++++++++++++++')
        self.fp = open('book.json', 'w', encoding='utf-8')



    # item就是yield后面的book对象
    def process_item(self, item, spider):
        # 以下这种模式不推荐    因为每传递过来一个对象 就会打开一次文件 对文件的操作过于频繁
        # # (1) write 方法必须要写一个字符串，而不能是其他对象
        # # (2) w模式 会每个对象都打开一次文件 覆盖之前的内容
        # with open('book.json', 'a', encoding='utf-8') as fp:
        #     fp.write(str(item))

        # 推荐使用以下模式 在前面定义
        self.fp.write(str(item))

        return item
    

    # 在爬虫文件结束之后就执行的方法
    def close_spider(self, spider):
        self.fp.close()